# CHPASE GATE!

New entry point. Should be easier to keep track of parameters and namespace issues using this new method

## Workflows

### Abstracted Workflow

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
# sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')
import yaml

from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from Circuit_Objs.qchard_coupobj import *
from Circuit_Objs.qchard_evolgates import *
from workflow_funcs import *

#### Fluxonium-IdealGridium

In [ ]:
fluxonium = Fluxonium(**light_fluxonium_params, **std_fluxonium_sim_params)
gridium = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
gridium.nlev = 8
fluxonium.nlev = 8

pulse_path = 'yamls/pulses/fluxonium_idealgridium_soft.yaml'
syscfg_path = 'yamls/syscfgs/fluxonium_idealgridium_soft.yaml'

fig = solve_coupled_qubits(fluxonium, gridium, pulse_path=pulse_path, syscfg_path=syscfg_path, n_shown_states=3)
fig.show()

It seems as if the rate limiting step for simulations that have high ending fidelity is actually simulating the propegator for all combinations of levels (typically 10x10). This does not seem to meaningfully change the graphs, so for trial simulations, truncated nlevs seems to be appropriate. For very complex/large qubit simulations, best practice is to first simulate the eigenvectors/eigenvalues before then saving the object. This saves sucessive solve times for later.



### Modular Workflow

When you want to get more in the weeds. Basically the contents of solve_coupled_qubits

#### SCQTransmon - Soft IdealGridium

In [ ]:
import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')

import yaml

from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from workflow_funcs import *

pulse_path = 'yamls/pulses/scqtransmon_idealgridium_soft.yaml'
syscfg_path = 'yamls/syscfgs/scqtransmon_idealgridium_soft.yaml'

with open(pulse_path, 'r') as f:
    data = yaml.safe_load(f)
    pulse_cfg = PulseConfig(**data)

with open(syscfg_path, 'r') as f:
    data = yaml.safe_load(f)
    system_cfg = SystemConfig(**data)
    del data

gridium = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
transmon = SCQTransmon(0.3,15,0,8,20)
gridium.nlev = 8
transmon.nlev = 8

qubitA = transmon
qubitB = gridium

n_shown_states = 3

qubitA, qubitB = load_qubits(qubitA, qubitB)
qubitA, qubitB = scale_qubitA_transition(qubitA, qubitB, system_cfg)
system = couple_qubits(qubitA, qubitB, system_cfg, pulse_cfg, mute=False)
t_points, U_t, phase_accum, fidelity = solve(system, pulse_cfg, system_cfg, solve_method='propagator', mute=False)
print(U_t)
fig = visualize_state_propagation(system, system_cfg, t_points, U_t, phase_accum, fidelity, n_shown_states=n_shown_states)

#### SCQTransmon - Hard IdealGridium

In [ ]:
import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')

import yaml

from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from workflow_funcs import *

pulse_path = 'yamls/pulses/scqtransmon_idealgridium_hard.yaml'
syscfg_path = 'yamls/syscfgs/scqtransmon_idealgridium_hard.yaml'

with open(pulse_path, 'r') as f:
    data = yaml.safe_load(f)
    pulse_cfg = PulseConfig(**data)

with open(syscfg_path, 'r') as f:
    data = yaml.safe_load(f)
    system_cfg = SystemConfig(**data)
    del data

gridium = IdealGridium(**hard_IdealGridium_params, **std_IdealGridium_sim_params)
transmon = SCQTransmon(0.3,15,0,8,20)
gridium.nlev_lc = 1000
gridium.nlev = 8
transmon.nlev = 8

qubitA = transmon
qubitB = gridium

n_shown_states = 3

qubitA, qubitB = load_qubits(qubitA, qubitB)
qubitA, qubitB = scale_qubitA_transition(qubitA, qubitB, system_cfg)
system = couple_qubits(qubitA, qubitB, system_cfg, pulse_cfg, mute=False)
t_points, U_t, phase_accum, fidelity = solve(system, pulse_cfg, system_cfg, solve_method='propagator', mute=False)
fig = visualize_state_propagation(system, system_cfg, t_points, U_t, phase_accum, fidelity, n_shown_states=n_shown_states)

#### SCQTransmon-Fluxonium

In [ ]:
import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')

import yaml
import numpy as np
from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from workflow_funcs import *

pulse_path = 'yamls/pulses/scqtransmon_heavy_fluxonium.yaml'
syscfg_path = 'yamls/syscfgs/scqtransmon_heavy_fluxonium.yaml'

with open(pulse_path, 'r') as f:
    data = yaml.safe_load(f)
    pulse_cfg = PulseConfig(**data)

with open(syscfg_path, 'r') as f:
    data = yaml.safe_load(f)
    system_cfg = SystemConfig(**data)
    del data

# Taken from https://arxiv.org/abs/2206.06203v2
transmon = SCQTransmon(0.2558, 10.5319, 0, 8, 30)
fluxonium = Fluxonium(0.5, 1, 8, np.pi, 8, 100) # Drive should be on this one (qubitB)

qubitA = transmon
qubitB = fluxonium
n_shown_states = 3

qubitA, qubitB = load_qubits(qubitA, qubitB)
qubitA, qubitB = scale_qubitA_transition(qubitA, qubitB, system_cfg)
system = couple_qubits(qubitA, qubitB, system_cfg, pulse_cfg, mute=False) # Coupling should be capacitive
t_points, U_t, phase_accum, fidelity = solve(system, pulse_cfg, system_cfg, solve_method='propagator', mute=False)
fig = visualize_state_propagation(system, system_cfg, t_points, U_t, phase_accum, fidelity, n_shown_states=n_shown_states)

In [ ]:
transmon = SCQTransmon(0.2558, 10.5319, 0, 8, 30)
zero1 = transmon.level(1)-transmon.level(0)
one2 = transmon.level(2)-transmon.level(1)
anharm = one2 - zero1
print(anharm)
print(zero1)

#### Light Fluxonium - Soft IdealGridium

In [ ]:
import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')
import yaml

from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from workflow_funcs import *

pulse_path = 'yamls/pulses/fluxonium_idealgridium_soft.yaml'
syscfg_path = 'yamls/syscfgs/fluxonium_idealgridium_soft.yaml'

with open(pulse_path, 'r') as f:
    data = yaml.safe_load(f)
    pulse_cfg = PulseConfig(**data)

with open(syscfg_path, 'r') as f:
    data = yaml.safe_load(f)
    system_cfg = SystemConfig(**data)
    del data

gridium = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
fluxonium = Fluxonium(**light_fluxonium_params, **std_fluxonium_sim_params)
gridium.nlev = 8
fluxonium.nlev = 8

qubitA = fluxonium
qubitB = gridium
n_shown_states = 3

qubitA, qubitB = load_qubits(qubitA, qubitB)
qubitA, qubitB = scale_qubitA_transition(qubitA, qubitB, system_cfg)
system = couple_qubits(qubitA, qubitB, system_cfg, pulse_cfg, mute=False)
t_points, U_t, phase_accum, fidelity = solve(system, pulse_cfg, system_cfg, solve_method='propagator', mute=False)
fig = visualize_state_propagation(system, system_cfg, t_points, U_t, phase_accum, fidelity, n_shown_states=n_shown_states)

In [ ]:
# xopt, cphase_pi_error_value = converge_on_pi(system, pulse_cfg, system_cfg)

In [ ]:
# xopt, infidelity = minimize_infidelity(system, pulse_cfg, system_cfg)

#### Light Fluxonium - Hard IdealGridium

In [ ]:
import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')
import yaml

from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from workflow_funcs import *

pulse_path = 'yamls/pulses/fluxonium_idealgridium_hard.yaml'
syscfg_path = 'yamls/syscfgs/fluxonium_idealgridium_hard.yaml'

with open(pulse_path, 'r') as f:
    data = yaml.safe_load(f)
    pulse_cfg = PulseConfig(**data)

with open(syscfg_path, 'r') as f:
    data = yaml.safe_load(f)
    system_cfg = SystemConfig(**data)
    del data

gridium = IdealGridium(**hard_IdealGridium_params, **std_IdealGridium_sim_params)
fluxonium = Fluxonium(**light_fluxonium_params, **std_fluxonium_sim_params)
gridium.nlev_lc = 1000
gridium.nlev = 8
fluxonium.nlev = 8

qubitA = fluxonium
qubitB = gridium
n_shown_states = 3

qubitA, qubitB = load_qubits(qubitA, qubitB)
qubitA, qubitB = scale_qubitA_transition(qubitA, qubitB, system_cfg)
system = couple_qubits(qubitA, qubitB, system_cfg, pulse_cfg, mute=False)
t_points, U_t, phase_accum, fidelity = solve(system, pulse_cfg, system_cfg, solve_method='propagator', mute=False)
fig = visualize_state_propagation(system, system_cfg, t_points, U_t, phase_accum, fidelity, n_shown_states=n_shown_states)

## Saving Diagonalized Objects

For speed in later use

In [ ]:
# gridium = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
# gridium.nlev = 10
# gridium.levels(eigvecs=True)
# gridium.save_obj('/Users/thomasersevim/QNL/2q_gridium/etc/qubits/')

# fluxonium = Fluxonium(**fluxonium_params, **std_fluxonium_sim_params)
# fluxonium.nlev = 10
# fluxonium.levels(eigvecs=True)
# fluxonium.save_obj('/Users/thomasersevim/QNL/2q_gridium/etc/qubits/')

# qubitA.levels(eigvecs=True)
# qubitA.save_obj('/Users/thomasersevim/QNL/2q_gridium/etc/qubits/')
qubitB.levels(eigvecs=True)
qubitB.save_obj('/Users/thomasersevim/QNL/2q_gridium/etc/qubits/')

## Testing

In [ ]:
transmon = SCQTransmon(0.3, 15, 0, 6, 8)
transmon.nlev_lc = 1000
transmon.nlev = 20
print(transmon.transition_energies())